In [14]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

realizamos la peticion a la web

In [15]:
result= requests.get("https://www.meteored.mx/clima_Puebla-America+Norte-Mexico-Puebla--1-21044.html")

Obtenemos estatusCode y html en un string

In [16]:
statusCode=result.status_code
htmlText=result.text

In [17]:
statusCode

200

pasamos el contenido html a un objeto BeautifulSoup

In [18]:
html=BeautifulSoup(result.text, "html.parser")

filtramos a entradas el contenido del objeto que esta en la etiquet 'td' y la clase 'first-column'

In [19]:
entradas=html.find_all('td',{'class': 'first-column'})

De lo que obtuvimos filtramos para la etiqueta 'dd' en la clase 'ddD2', unicamente para la hora actual, en este caso es la pocision 0.
recorremos con un for cada linea obteniendo el texto.

In [7]:
primeracol = ["Puebla","14:00"]

In [8]:
en=entradas[0].find_all('dd', {'class': 'ddD2'})
aux=0
for aux2 in  en:
    primeracol.append(aux2.getText())
    


procedemos a sacar la segunda parte de los datos

In [10]:
entradas=html.find_all('td',{'class': 'second-column'})
en=entradas[0].find_all('dd', {'class': 'ddD2'})
aux=0
for aux2 in  en:
    primeracol.append(aux2.getText())
    
primeracol

['Puebla',
 '14:00',
 '5 FPS: 6-10',
 '34%',
 '54%',
 '5 km/h',
 '1014 hPa',
 '35 km',
 '25 °C',
 '7 °C',
 'No',
 '35 km/h',
 '4800 m']

Procedemos a sacar la direccion del viento que esta en otro apartado

In [42]:
viento=html.find_all('span',{'class': 'datos-viento'})
viento=viento[0].find_all('strong')
viento[0].getText()

'Sureste'

limpieza de datos

In [43]:
fin=[primeracol[0]]
fin.append(primeracol[1])
#separamos primera linea
aux= primeracol[2].split(" FPS: ")
fin.append(aux[0])
#primer indice
if (aux[1] == 'no'):
        fin.append("0")
else:
    fin.append(aux[1])
#eliminamos ultimo caracter
fin.append(primeracol[3][0:len(primeracol[3])-1])
fin.append(primeracol[4][0:len(primeracol[4])-1])

#separamos por espacio y guardamos primer token
fin.append((primeracol[5].split(" "))[0])
fin.append((primeracol[6].split(" "))[0])
fin.append((primeracol[7].split(" "))[0])
fin.append((primeracol[8].split(" "))[0])
fin.append((primeracol[9].split(" "))[0])

#identificamos si es numero o 'no'
if (primeracol[10] == 'No'):
        fin.append("0")
else:
    fin.append(primeracol[10])

#separamos por espacio
fin.append((primeracol[11].split(" "))[0])
fin.append((primeracol[12].split(" "))[0])
fin

['Puebla',
 '14:00',
 '5',
 '6-10',
 '34',
 '54',
 '5',
 '1014',
 '35',
 '25',
 '7',
 '0',
 '35',
 '4800']

guardamos en una base de datos

In [44]:
#abrimos la tabla
frame=pd.read_csv('auxx.csv')
frame

,Lugar,Hora,Indice UV,Algo,Humedad,Nubosidad,Viento-Medio,Presion,Visibilidad,Sensacion Termica,Punto de rocio,Niebla,Viento_racha,Cota Nieve,Viento_Medio
0,Puebla,14:00,2,0,28,37,NaN,1012,40,25,5,0,30,4900,15
1,Puebla,14:00,2,0,28,37,NaN,1012,40,25,5,0,30,4900,15
2,Puebla,14:00,2,0,28,37,NaN,1012,40,25,5,0,30,4900,15
3,Puebla,14:00,0,0,43,39,NaN,1016,35,18,6,0,32,4700,14
4,Puebla,14:00,2,0,22,42,NaN,1014,40,25,2,0,40,4800,19


In [45]:
#creamoso una table igual para despues concatenar
frame2=pd.DataFrame([fin],columns=["Lugar","Hora","Indice UV","Algo","Humedad","Nubosidad","Viento_Medio","Presion","Visibilidad","Sensacion Termica","Punto de rocio","Niebla","Viento_racha","Cota Nieve"])

In [46]:
frame2

,Lugar,Hora,Indice UV,Algo,Humedad,Nubosidad,Viento_Medio,Presion,Visibilidad,Sensacion Termica,Punto de rocio,Niebla,Viento_racha,Cota Nieve
0,Puebla,14:00,5,6-10,34,54,5,1014,35,25,7,0,35,4800


In [47]:
#concatenamos
res=frame.append(frame2,sort=False)


In [48]:
#guardamos
res.to_csv('auxx.csv', header=True, index=False)

['Puebla',
 '14:00',
 '2',
 '0',
 '22',
 '42',
 '19',
 '1014',
 '40',
 '25',
 '2',
 '0',
 '40',
 '4800']

,Lugar,Hora,Indice UV,Algo,Humedad,Nubosidad,Viento-Medio,Presion,Visibilidad,Sensacion Termica,Punto de rocio,Niebla,Viento_racha,Cota Nieve,Viento_Medio
0,Puebla,14:00,2,0,28,37,NaN,1012,40,25,5,0,30,4900,15
1,Puebla,14:00,2,0,28,37,NaN,1012,40,25,5,0,30,4900,15
2,Puebla,14:00,2,0,28,37,NaN,1012,40,25,5,0,30,4900,15
3,Puebla,14:00,0,0,43,39,NaN,1016,35,18,6,0,32,4700,14
